In [ ]:
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [3]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog

In [4]:
import time
import numpy as np
import cv2

In [ ]:
file_path = '/content/'
file_name = 'running2.mp4'
file_1 = file_name.split(".")
print(file_1[1])
file = file_path+file_name

video = cv2.VideoCapture(file)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

print("video width", width)
print("video_height",height)
print("frames per second",frames_per_second)
print(num_frames)

In [29]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)


In [30]:
v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

In [31]:
def video_process(video, total_frames):

    frame_rate = 0
    while True:
        true, frame = video.read()
        if not true:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        pred_frame = predictor(frame)

        panoptic_seg, segments_info = predictor(frame)["panoptic_seg"]
        output = v.draw_panoptic_seg_predictions(frame, panoptic_seg.to("cpu"), segments_info)

        output = cv2.cvtColor(output.get_image(), cv2.COLOR_RGB2BGR)

        frame_rate += 1

        yield output, frame_rate
        
        if frame_rate > total_frames:
            break

        

In [ ]:
total_frames = 100 #adjust total_frames to your desired length if you want full length of video enter "num_frames"

output_video_name = file_path+file_1[0]+"_output_video"+"."+file_1[1]
output_image_name = file_path+file_1[0]+"_output_image"+".png"

frame_to_write = 99 #choose which frame you would want to write as an image

video_writer = cv2.VideoWriter(output_video_name, fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

for output,frame_rate in video_process(video, total_frames):
    
    print("Processing frame ===",frame_rate,"Out of ",total_frames)
    if frame_rate == frame_to_write: #write the frame

      print("Writing frame == ",frame_rate)
      cv2.imwrite(output_image_name, output)

    video_writer.write(output)


print("Total frames = ",num_frames)
print("Number of frames processed = ",total_frames)

video.release()
video_writer.release()
cv2.destroyAllWindows()